# Convert raw data to JSON for package

In [8]:
import pandas as pd
df_nrg=pd.read_csv("world_energy_mix.csv")
df_nrg

,Unnamed: 0,Year,Fossil fuels electricity generation,Geothermal electricity generation,Hydroelectricity generation,Nuclear power generation,Solar electricity generation,Wind electricity generation,country_id,country_name
0,0,1980,0.23100,0.0,0.711,0.0,0.000,0.0,107,Afghanistan
1,1,1981,0.27100,0.0,0.721,0.0,0.000,0.0,107,Afghanistan
2,2,1982,0.24500,0.0,0.707,0.0,0.000,0.0,107,Afghanistan
3,3,1983,0.25400,0.0,0.746,0.0,0.000,0.0,107,Afghanistan
4,4,1984,0.26200,0.0,0.757,0.0,0.000,0.0,107,Afghanistan
...,...,...,...,...,...,...,...,...,...,...
7632,7632,2015,4.31178,0.0,4.940,0.0,0.008,0.0,106,Zimbabwe
7633,7633,2016,3.71488,0.0,2.955,0.0,0.009,0.0,106,Zimbabwe
7634,7634,2017,3.21856,0.0,3.929,0.0,0.011,0.0,106,Zimbabwe
7635,7635,2018,3.93202,0.0,5.000,0.0,0.014,0.0,106,Zimbabwe


In [5]:
df_p = pd.read_csv("https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv")

df_p.columns

Index(['FIFA', 'Dial', 'ISO3166-1-Alpha-3', 'MARC', 'is_independent',
       'ISO3166-1-numeric', 'GAUL', 'FIPS', 'WMO', 'ISO3166-1-Alpha-2', 'ITU',
       'IOC', 'DS', 'UNTERM Spanish Formal', 'Global Code',
       'Intermediate Region Code', 'official_name_fr', 'UNTERM French Short',
       'ISO4217-currency_name', 'Developed / Developing Countries',
       'UNTERM Russian Formal', 'UNTERM English Short',
       'ISO4217-currency_alphabetic_code',
       'Small Island Developing States (SIDS)', 'UNTERM Spanish Short',
       'ISO4217-currency_numeric_code', 'UNTERM Chinese Formal',
       'UNTERM French Formal', 'UNTERM Russian Short', 'M49',
       'Sub-region Code', 'Region Code', 'official_name_ar',
       'ISO4217-currency_minor_unit', 'UNTERM Arabic Formal',
       'UNTERM Chinese Short', 'Land Locked Developing Countries (LLDC)',
       'Intermediate Region Name', 'official_name_es', 'UNTERM English Formal',
       'official_name_cn', 'official_name_en', 'ISO4217-currency_count

In [136]:
df_country = df_p[['ISO3166-1-Alpha-3', 'CLDR display name', 'official_name_en', 'Region Name']]
df_country.columns = ['iso_code', 'label', 'official_name_en', 'region']

In [137]:
df = df_nrg[["country_name"]]

In [138]:
df = df.drop_duplicates(keep='first', inplace=False, ignore_index=True)
len(df)

190

In [139]:
len(df_country)

250

In [140]:
def fix_name(df):
    df.replace("Antigua and Barbuda", "Antigua & Barbuda",inplace=True)
    df.replace("Bosnia and Herzegovina", "Bosnia",inplace=True)
    df.replace("Burma (Myanmar)", "Myanmar",inplace=True)
    df.replace("Democratic Republic of the Congo", "Congo - Kinshasa",inplace=True)
    df.replace("Ivory Coast", "Côte d’Ivoire",inplace=True)
    df.replace("Macao", "Macau",inplace=True)
    df.replace("Republic of the Congo", "Congo - Brazzaville",inplace=True)
    df.replace("Saint Lucia", "St. Lucia",inplace=True)
    df.replace("Saint Vincent and the Grenadines", "St. Vincent & Grenadines",inplace=True)
    df.replace("Sao Tome and Principe", "São Tomé & Príncipe",inplace=True)
    df.replace("Swaziland", "Eswatini",inplace=True)
    df.replace("Trinidad and Tobago", "Trinidad & Tobago",inplace=True)
    df.replace("United Kingdom", "UK",inplace=True)
    df.replace("USA", "US",inplace=True)
    return df

In [141]:
df = fix_name(df)
df_nrg = fix_name(df_nrg)

In [142]:
df_join = df.merge(df_country, how="left", left_on='country_name', right_on='label', indicator=True)

In [143]:
assert len(df_join.query("_merge == 'left_only'")) == 0

In [144]:
df_join

,country_name,iso_code,label,official_name_en,region,_merge
0,Afghanistan,AFG,Afghanistan,Afghanistan,Asia,both
1,Albania,ALB,Albania,Albania,Europe,both
2,Algeria,DZA,Algeria,Algeria,Africa,both
3,Angola,AGO,Angola,Angola,Africa,both
4,Antigua & Barbuda,ATG,Antigua & Barbuda,Antigua and Barbuda,Americas,both
...,...,...,...,...,...,...
185,Venezuela,VEN,Venezuela,Venezuela (Bolivarian Republic of),Americas,both
186,Vietnam,VNM,Vietnam,Viet Nam,Asia,both
187,Yemen,YEM,Yemen,Yemen,Asia,both
188,Zambia,ZMB,Zambia,Zambia,Africa,both


In [145]:
df_final = df_nrg.merge(df_join[['country_name', 'iso_code', 'official_name_en', 'region']], how="left", left_on='country_name', right_on='country_name', indicator=True)


In [146]:
assert len(df_final.query("_merge == 'left_only'")) == 0

In [147]:
df_final.drop(columns=['Unnamed: 0', '_merge'], inplace=True)
df_final.columns
df_final.columns=['year', 'fossil', 'geothermal', 'hydroelectricity',
       'nuclear', 'solar', 'wind', 'country_id', 'country_name', 'iso_code',
       'official_name_en', 'region']

In [148]:
df_final.query("wind > 337")

,year,fossil,geothermal,hydroelectricity,nuclear,solar,wind,country_id,country_name,iso_code,official_name_en,region
1523,2018,4707.118620,0.123500,1187.20801,272.17599,177.200000,365.800000,115,China,CHN,China,Asia
1524,2019,4812.964500,0.123500,1254.46460,330.12201,224.100000,405.700000,115,China,CHN,China,Asia
7356,2020,2419.230282,16.930105,291.11084,789.91864,132.630753,337.509815,48,US,USA,United States of America,Americas


In [153]:
# Taiwan has no region and no official_name_en in the dataset, let's fix this
df_final.loc[df_final.iso_code == 'TWN', ['official_name_en', 'region']] = 'Taiwan', 'Asia'

In [155]:
# df_final.query("iso_code == 'TWN'")

In [156]:
energy_mix = {}
for code in df_final.iso_code.unique():
#for code in ['FRA', 'AFG', 'VNM']:
    df=df_final.query("iso_code == @code")
    max_year = df.year.max()
    for year in range(max_year, 1980, -1):
        # print(year)
        df_tmp=df_final.query("iso_code == @code and year == @year")
        #print(df_tmp.isna().values.any())
        #print(df_tmp)
        if df_tmp.isnull().values.any() == False:
            energy_mix[code]=df_tmp.iloc[0].to_dict()
            energy_mix[code]["total"]=energy_mix[code]["fossil"]+energy_mix[code]["geothermal"]+\
                energy_mix[code]["hydroelectricity"]+energy_mix[code]["nuclear"]+\
                energy_mix[code]["solar"]+energy_mix[code]["wind"]
            break
    else:
        print("ERROR: missing values for all year for ", code)
        print(df)
    #print(df_tmp)
energy_mix['FRA']

{'year': 2020,
 'fossil': 44.553,
 'geothermal': 0.155753680282,
 'hydroelectricity': 60.75027,
 'nuclear': 335.41461,
 'solar': 14.1455621272,
 'wind': 43.5316709709,
 'country_id': 194,
 'country_name': 'France',
 'iso_code': 'FRA',
 'official_name_en': 'France',
 'region': 'Europe',
 'total': 498.55086677838193}

In [157]:
import json
with open("global_energy_mix.json", "w") as outfile:
    json.dump(energy_mix, outfile)